# YOLOv7 model Evaluation

The following code loads the YOLOv7 trained model and calculates evaluation metrics based on its perfromance against a testing set.

In [1]:
import sys
import torch
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import PIL
import seaborn as sns
import io
from torch.utils.tensorboard import SummaryWriter


yolov7_path = Path(r"C:\Users\kyled\yolov7\yolov7")
sys.path.append(str(yolov7_path))

from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
from utils.plots import plot_one_box

def load_model(model_path, device):
    model = attempt_load(model_path, map_location=device)
    model.to(device).eval()
    return model

def process_image(image_path, img_size=640):
    # Loading the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resizing and padding image
    h, w, _ = img.shape
    r = img_size / max(h, w) 
    new_w = int(round(w * r))
    new_h = int(round(h * r))
    img_resized = cv2.resize(img, (new_w, new_h))

    img_padded = np.full((img_size, img_size, 3), 128, dtype=np.uint8)
    img_padded[(img_size - new_h) // 2:(img_size - new_h) // 2 + new_h, 
               (img_size - new_w) // 2:(img_size - new_w) // 2 + new_w, :] = img_resized

    img_tensor = torch.from_numpy(img_padded).permute(2, 0, 1).float().unsqueeze(0)
    img_tensor /= 255.0
    return img_tensor

def detect_objects(model, img_tensor, device):
    # Performing object detection
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        pred = model(img_tensor)[0]
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)
    return pred

def draw_detections(original_img, img_tensor, detections, class_names, img_size=640):
    # Checking if detections are present
    if detections is not None:
        for i, det in enumerate(detections):
            if len(det):
                # Rescaling boxes to original image size
                det[:, :4] = scale_coords(img_tensor.shape[2:], det[:, :4], original_img.shape).round()

                
                for *xyxy, conf, cls in reversed(det):
                    label_with_conf = f'{class_names[int(cls)]} {conf:.2f}'
                    plot_one_box(xyxy, original_img, label=label_with_conf, color=(255, 0, 0), line_thickness=3)

    return original_img

def parse_label_file(label_path, img_shape):
    #Getting ground truth boxes
    ground_truth_boxes = []
    with open(label_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.split())
            x1 = int((x_center - width / 2) * img_shape[1])
            y1 = int((y_center - height / 2) * img_shape[0])
            x2 = int((x_center + width / 2) * img_shape[1])
            y2 = int((y_center + height / 2) * img_shape[0])
            ground_truth_boxes.append([x1, y1, x2, y2, class_id])
    return ground_truth_boxes

def calculate_iou(box1, box2):
    #Comparing the detected and true bounding boxes using IoU
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - intersection_area
    iou = intersection_area / union_area if union_area != 0 else 0

    return iou


def plot_confusion_matrix(confusion_matrix, class_names):
    # Creating a matrix from the confusion matrix data
    matrix = []
    for class_name in class_names:
        row = [confusion_matrix[class_name]['TP'], confusion_matrix[class_name]['FP'], confusion_matrix[class_name]['FN']]
        matrix.append(row)

    matrix = np.array(matrix)

    fig_size_width = max(8, len(class_names) * 0.5)
    fig_size_height = max(6, len(class_names) * 0.5)
    plt.figure(figsize=(fig_size_width, fig_size_height))

    ax = sns.heatmap(matrix, annot=True, fmt="d", cmap="Blues", linewidths=.5)

    ax.set_xlabel('Predicted labels', fontsize=12)
    ax.set_ylabel('True labels', fontsize=12)
    ax.set_title('Confusion Matrix', fontsize=16)
    ax.xaxis.set_ticklabels(['True Positive', 'False Positive', 'False Negative'], fontsize=10)
    ax.yaxis.set_ticklabels(class_names, fontsize=10, va='center', rotation = 0)

    plt.tight_layout()

    return plt.gcf()


class_names = ['pizza', 'Mushroom', 'Pepperoni', 'Yellow Peppers', 'Black Olives', 'Onion', 'Ham', 'Tomato', 'Broccoli', 'Green Olives']


log_dir = r"C:\Users\kyled\yolov7\runs\YOLOv7_tensorboard"
writer = SummaryWriter(log_dir)

def main():
    yolov7()    
    
def yolov7():    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_path = r"C:\Users\kyled\yolov7\runs\train\exp13\weights\best.pt"
    test_images_dir = r"C:\Users\kyled\Downloads\yolo_formatted_testing_set\images"
    model = load_model(model_path, device).to(device)

    # Initializing counters for overall metrics
    total_TP, total_FP, total_FN = 0, 0, 0

    # Initializing confusion matrix
    confusion_matrix = {class_name: {'TP': 0, 'FP': 0, 'FN': 0} for class_name in class_names}

    for image_path in glob.glob(test_images_dir + '/*.jpg'):
        original_img = cv2.imread(image_path)
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        img_tensor = process_image(image_path)

        detections = detect_objects(model, img_tensor.to(device), device)
        
        label_file = image_path.replace('images', 'Labels').replace('.jpg', '.txt')
        ground_truth_boxes = parse_label_file(label_file, original_img.shape)

        # Processing detections
        for det in detections[0]:
            det_box = [int(det[0]), int(det[1]), int(det[2]), int(det[3])]
            det_class = int(det[-1])

            # Checking if detection matches any ground truth box of the same class
            matched = False
            for gt_box in ground_truth_boxes:
                if det_class == gt_box[4] and calculate_iou(det_box, gt_box[:4]) > 0.5:
                    matched = True
                    break

            if matched:
                total_TP += 1
                confusion_matrix[class_names[det_class]]['TP'] += 1
            else:
                total_FP += 1
                confusion_matrix[class_names[det_class]]['FP'] += 1

        for gt_box in ground_truth_boxes:
            gt_class = int(gt_box[4])

            detected = False
            for det in detections[0]:
                det_box = [int(det[0]), int(det[1]), int(det[2]), int(det[3])]
                if gt_class == int(det[-1]) and calculate_iou(det_box, gt_box[:4]) > 0.5:
                    detected = True
                    break

            if not detected:
                total_FN += 1
                confusion_matrix[class_names[gt_class]]['FN'] += 1

    # Calculating overall Metrics
    precision = total_TP / (total_TP + total_FP) if total_TP + total_FP > 0 else 0
    recall = total_TP / (total_TP + total_FN) if total_TP + total_FN > 0 else 0
    F1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
        
        
    text_string = f"Precision: {precision}\nRecall: {recall}\nF1-Score: {F1}"
    writer.add_text('Metrics/Summary', text_string)
        
    writer.add_scalar('YOLOv7 Precision', precision)
    writer.add_scalar('YOLOv7 Recall', recall)
    writer.add_scalar('YOLOv7 F1-Score', F1)

    confusion_matrix_plot = plot_confusion_matrix(confusion_matrix, class_names)
    writer.add_figure('YOLOv7 Confusion Matrix', confusion_matrix_plot)

    writer.close()
        
if __name__ == '__main__':
    main()




Fusing layers... 
IDetect.fuse


C:\Users\kyled\anaconda3\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
import sys
import torch
import cv2
import numpy as np
from pathlib import Path
from PIL import Image

def detect_objects(model, img_tensor, device):
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        pred = model(img_tensor)[0]
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)
    
    class_indices = [int(detection[-1]) for detection in pred[0]] if pred[0] is not None else []
    return pred, class_indices

def analyze_detections(class_indices, class_names):
    # Counting the occurrences of each class
    class_counts = {class_name: 0 for class_name in class_names}
    for index in class_indices:
        class_name = class_names[index]
        class_counts[class_name] += 1
    return class_counts


def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Set the path to your trained model and test image
    model_path = r"C:\Users\kyled\yolov7\runs\train\exp13\weights\best.pt"
    test_image_path = r"C:\Users\kyled\Downloads\pizza_test.jpg"

    # Load the model
    model = load_model(model_path, device).to(device)

    # Load and process the image
    original_img = cv2.imread(test_image_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    img_tensor = process_image(test_image_path)

    # Detect objects
    detections, class_indices = detect_objects(model, img_tensor.to(device), device)

    # Analyze detections
    class_counts = analyze_detections(class_indices, class_names)

    # Print results
    print(f"Number of pizzas detected: {class_counts['pizza']}")
    print("Toppings count:")
    for topping, count in class_counts.items():
        if topping != 'pizza':
            print(f"  {topping}: {count}")

    # Draw and display detections
    detected_img = draw_detections(original_img, img_tensor, detections, class_names)
    Image.fromarray(detected_img).show()

if __name__ == '__main__':
    main()

Fusing layers... 
IDetect.fuse
Number of pizzas detected: 2
Toppings count:
  Mushroom: 0
  Pepperoni: 22
  Yellow Peppers: 0
  Black Olives: 0
  Onion: 0
  Ham: 0
  Tomato: 0
  Broccoli: 0
  Green Olives: 0
